In [14]:
from recordclass import dataobject, make_dataclass, enable_gc, litelist
from random import randint

In [15]:
import tracemalloc
class Tracer:
    def __enter__(self):
        if tracemalloc.is_tracing():
            raise ValueError('nesting tracemalloc is not allowed')
        self.allocated = None
        tracemalloc.start()
        return self
    def __exit__(self, exc_type, exc_value, exc_traceback):
        current, peak = tracemalloc.get_traced_memory()
        tracemalloc.stop()
        self.allocated = current

In [16]:
N = 24
print(2**N/1000000, "млн.")

16.777216 млн.


### Binary Tree

In [17]:
class Tree(dataobject):
   __fields__ = 'root',

class Node(dataobject): 
    __fields__ = 'left', 'right'

class Leaf(dataobject):
    __fields__ = 'value',

In [24]:
def add_nodes(depth):
    if depth == 0:
        return Leaf(randint(0,10))
    return Node(add_nodes(depth-1), add_nodes(depth-1))

In [19]:
@enable_gc
class TreeGC(dataobject):
   __fields__ = 'root',

@enable_gc
class NodeGC(dataobject): 
    __fields__ = 'left', 'right'

@enable_gc
class LeafGC(dataobject):
    __fields__ = 'value',

In [25]:
def add_nodes_gc(depth):
    if depth == 0:
        return LeafGC(randint(0,10))
    return NodeGC(add_nodes_gc(depth-1), add_nodes_gc(depth-1))

In [26]:
class TreeSlots:
    __slots__ = 'root',
    def __init__(self, root):
        self.root = root

class NodeSlots:
    __slots__ = 'left', 'right'
    def __init__(self, left, right):
        self.left = left
        self.right = right
    
class LeafSlots:
    __slots__ = 'value',
    def __init__(self, value):
        self.value = value

In [27]:
def add_nodes_slots(depth):
    if depth == 0:
        return LeafSlots(randint(0,10))
    return NodeSlots(add_nodes_slots(depth-1), add_nodes_slots(depth-1))

In [28]:
with Tracer() as t1:
   tree = Tree(add_nodes(N))
print("dataobject:", t1.allocated // 1_000_000, 'Mb')
del tree, t1

dataobject: 939 Mb


In [29]:
%time tree = Tree(add_nodes(N))
%time del tree

CPU times: user 22 s, sys: 272 ms, total: 22.3 s
Wall time: 22.3 s
CPU times: user 601 ms, sys: 68.1 ms, total: 669 ms
Wall time: 668 ms


In [30]:
with Tracer() as t2:
   tree_slots = TreeSlots(add_nodes_slots(N))
print("__slots__:", t2.allocated // 1_000_000, 'Mb')
del tree_slots, t2

__slots__: 2013 Mb


In [31]:
%time tree_slots = TreeSlots(add_nodes_slots(N))
%time del tree_slots

CPU times: user 1min 1s, sys: 532 ms, total: 1min 2s
Wall time: 1min 2s
CPU times: user 839 ms, sys: 84.1 ms, total: 923 ms
Wall time: 921 ms


In [32]:
with Tracer() as t3:
   tree_gc = TreeGC(add_nodes_gc(N))
print("dataobject+gc:", t3.allocated // 1000000, 'Mb')
del tree_gc, t3

dataobject+gc: 2013 Mb


In [33]:
%time tree_gc = TreeGC(add_nodes_gc(N))
%time del tree_gc

CPU times: user 1min 1s, sys: 640 ms, total: 1min 1s
Wall time: 1min 1s
CPU times: user 670 ms, sys: 104 ms, total: 774 ms
Wall time: 773 ms


In [34]:
import sys
print('Size per instance:')
print('   node:', sys.getsizeof(Node(None,None)),
                  sys.getsizeof(NodeSlots(None,None)), 
                  sys.getsizeof(NodeGC(None,None)))
print('   leaf:', sys.getsizeof(Leaf(None)),
                  sys.getsizeof(LeafSlots(None)),
                  sys.getsizeof(LeafGC(None)))

Size per instance:
   node: 32 64 64
   leaf: 24 56 56


### General Tree

In [35]:
class GTree(dataobject):
   __fields__ = 'root',

class GNode(dataobject): 
    __fields__ = 'children',

class GLeaf(dataobject):
    __fields__ = 'value',

In [36]:
def add_gnodes(depth):
    if depth == 0:
        return GLeaf(randint(0,10))
    children = litelist([])
    gnode = GNode(children)
    children.append(add_gnodes(depth-1))
    children.append(add_gnodes(depth-1))
    children.trim()
    return gnode

In [37]:
with Tracer() as gt1:
   gtree = GTree(add_gnodes(20))
print("dataobject:", gt1.allocated // 1000000, 'Mb')
del gtree, gt1

dataobject: 117 Mb


In [38]:
%time gtree = GTree(add_gnodes(20))
del gtree

CPU times: user 1.64 s, sys: 28 ms, total: 1.67 s
Wall time: 1.67 s


In [39]:
class GTreeSlots:
    __slots__ = 'root',
    def __init__(self, root):
        self.root = root

class GNodeSlots:
    __slots__ = 'children', 
    def __init__(self, children):
        self.children = children
    
class GLeafSlots:
    __slots__ = 'value',
    def __init__(self, value):
        self.value = value

In [40]:
def add_gnodes_slots(depth):
    if depth == 0:
        return GLeafSlots(randint(0,10))
    children = []
    gnode = GNodeSlots(children)
    children.append(add_gnodes_slots(depth-1))
    children.append(add_gnodes_slots(depth-1))
    return gnode

In [41]:
with Tracer() as gt2:
   gtree_slots = GTreeSlots(add_gnodes_slots(20))
print("__slots__:", gt2.allocated // 1000000, 'Mb')
del gtree_slots, gt2

__slots__: 226 Mb


In [42]:
%time gtree_slots = GTreeSlots(add_gnodes_slots(20))
del gtree_slots

CPU times: user 5.92 s, sys: 87.9 ms, total: 6.01 s
Wall time: 6 s
